# LSTM Network - Stock Price Prediction

A stacked LSTM (Long Short-Term Memory) model built using data from a course on Udemy called [Deep Learning A-Z](https://www.udemy.com/course/deeplearning/learn/lecture/8374794). The dataset consists of 